In [69]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [70]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [71]:
tf.keras.backend.clear_session()

In [72]:
def add_cos_monthly(df, column_name):
    values = [np.cos(2 * np.pi * (i % 12) / 12) for i in range(len(df))]
    df[column_name] = values
    return df

In [73]:
subject = 'Espírito Santo - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [74]:
data = pd.read_csv('2003_mo_model_input_ES.csv')
data = add_cos_monthly(data, "Sazonalidade")
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Espírito Santo - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Espírito Santo - IDH,Espírito Santo - PIB - Estadual,Espírito Santo - PIB - Construção Civil,Espírito Santo - PIB - Per Capita,Espírito Santo - PIB - Preços de Mercado,Espírito Santo - Produção de Cimento (t),Espírito Santo - value,Sazonalidade,Espírito Santo - Consumo de Cimento (t)
0,2003-1,8.177444,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,0.760059,6.342537e+07,3.523987e+06,16.799308,5.948249e+07,98.171240,0.268445,1.000000e+00,48.426
1,2003-2,8.171622,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,0.760222,6.350834e+07,3.527412e+06,16.807398,5.951949e+07,97.897317,0.270512,8.660254e-01,54.969
2,2003-3,8.165799,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,0.760385,6.359131e+07,3.530838e+06,16.815488,5.955649e+07,98.070539,0.271941,5.000000e-01,53.847
3,2003-4,8.159977,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,0.760548,6.367428e+07,3.534263e+06,16.823577,5.959349e+07,98.460789,0.273321,6.123234e-17,50.375
4,2003-5,8.154154,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,0.760710,6.375725e+07,3.537689e+06,16.831667,5.963048e+07,99.084134,0.274453,-5.000000e-01,52.576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.919726,0.505935,-8.660254e-01,100.170
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.756019,0.504835,-5.000000e-01,85.832
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.684567,0.502971,-1.836970e-16,91.318
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.664258,0.500670,5.000000e-01,77.187


In [75]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Espírito Santo - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Espírito Santo - IDH,Espírito Santo - PIB - Estadual,Espírito Santo - PIB - Construção Civil,Espírito Santo - PIB - Per Capita,Espírito Santo - PIB - Preços de Mercado,Espírito Santo - Produção de Cimento (t),Espírito Santo - value,Sazonalidade
0,-0.758586,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-2.007957,-1.927365,-0.943195,0.087180,-1.813287,-1.319688,-1.294991,1.414214e+00
1,-0.763437,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.978578,-1.900219,-0.892433,0.126806,-1.716680,-1.326255,-1.246951,1.224745e+00
2,-0.768288,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.949198,-1.873074,-0.841670,0.166431,-1.620072,-1.322102,-1.213715,7.071068e-01
3,-0.773139,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.919818,-1.845928,-0.790908,0.206056,-1.523465,-1.312746,-1.181649,1.514512e-16
4,-0.777990,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.890438,-1.818783,-0.740145,0.245681,-1.426858,-1.297802,-1.155335,-7.071068e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.925704,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.304933,1.073521,-0.934891,-1.155565,-0.875236,-1.727603,1.093145,-1.224745e+00
188,0.920658,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.292490,1.056445,-0.903403,-1.167749,-0.908951,-1.753915,1.108075,-7.071068e-01
189,0.915611,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.280047,1.039370,-0.871915,-1.179933,-0.942665,-1.778143,1.130399,-1.949312e-16
190,0.910564,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.267603,1.022295,-0.840427,-1.192116,-0.976380,-1.806324,1.153917,7.071068e-01


In [76]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      48.646
1      43.597
2      56.331
3      48.488
4      52.856
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Espírito Santo - Consumo de Cimento (t), Length: 240, dtype: float64

In [77]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Espírito Santo - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Espírito Santo - IDH,Espírito Santo - PIB - Estadual,Espírito Santo - PIB - Construção Civil,Espírito Santo - PIB - Per Capita,Espírito Santo - PIB - Preços de Mercado,Espírito Santo - Produção de Cimento (t),Espírito Santo - value,Sazonalidade
0,-0.758586,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-2.007957,-1.927365,-0.943195,0.087180,-1.813287,-1.319688,-1.294991,1.414214e+00
1,-0.763437,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.978578,-1.900219,-0.892433,0.126806,-1.716680,-1.326255,-1.246951,1.224745e+00
2,-0.768288,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.949198,-1.873074,-0.841670,0.166431,-1.620072,-1.322102,-1.213715,7.071068e-01
3,-0.773139,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.919818,-1.845928,-0.790908,0.206056,-1.523465,-1.312746,-1.181649,1.514512e-16
4,-0.777990,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.890438,-1.818783,-0.740145,0.245681,-1.426858,-1.297802,-1.155335,-7.071068e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.538868,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,1.408727,1.007109,-1.505857,-1.639139,-1.626063,-0.138284,1.037783,1.224745e+00
158,1.502037,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,1.408716,1.017654,-1.496536,-1.616758,-1.583348,-0.199466,1.039895,7.071068e-01
159,1.465205,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,1.408704,1.028199,-1.487214,-1.594377,-1.540633,-0.263743,1.042387,1.514512e-16
160,1.428374,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,1.408692,1.038744,-1.477892,-1.571996,-1.497917,-0.329315,1.044800,-7.071068e-01


In [78]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0       48.646
1       43.597
2       56.331
3       48.488
4       52.856
        ...   
157     77.958
158    105.434
159     86.699
160     90.574
161     85.148
Name: Espírito Santo - Consumo de Cimento (t), Length: 162, dtype: float64

In [79]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [80]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 16)

In [81]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Espírito Santo - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Espírito Santo - IDH,Espírito Santo - PIB - Estadual,Espírito Santo - PIB - Construção Civil,Espírito Santo - PIB - Per Capita,Espírito Santo - PIB - Preços de Mercado,Espírito Santo - Produção de Cimento (t),Espírito Santo - value,Sazonalidade
126,0.929165,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,0.503951,0.864276,-0.433841,-0.655804,0.094697,1.142222,0.875311,-1.414214e+00
127,0.963455,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,0.514753,0.876404,-0.493112,-0.692821,0.038864,1.099243,0.884376,-1.224745e+00
128,0.997745,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,0.525554,0.888532,-0.552384,-0.729838,-0.016968,1.049585,0.894352,-7.071068e-01
129,1.032035,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,0.536356,0.900660,-0.611655,-0.766855,-0.072801,1.007713,0.904024,-1.949312e-16
130,1.066325,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,0.547157,0.912788,-0.670927,-0.803872,-0.128633,0.957549,0.912127,7.071068e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.925704,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.304933,1.073521,-0.934891,-1.155565,-0.875236,-1.727603,1.093145,-1.224745e+00
188,0.920658,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.292490,1.056445,-0.903403,-1.167749,-0.908951,-1.753915,1.108075,-7.071068e-01
189,0.915611,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.280047,1.039370,-0.871915,-1.179933,-0.942665,-1.778143,1.130399,-1.949312e-16
190,0.910564,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.267603,1.022295,-0.840427,-1.192116,-0.976380,-1.806324,1.153917,7.071068e-01


In [82]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 16)

In [83]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [84]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(180, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(36,activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
#                         validation_split=0.1,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [85]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [86]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[2910208350, 2779808283, 1458226475, 1967453755, 2629801941, 535696234, 3327005801, 1759783473, 432292446, 4151389751, 2813077823, 1328631124, 3595439023, 2086591015, 2813165187, 3625930937, 372339926, 1296447139, 1191732682, 599881116, 968969988, 3290526518, 3187067184, 1268313686, 2678376914, 1465489110, 949683433, 3371338609, 907666151, 625529532, 1099672765, 3258230505, 772579197, 3324487816, 3728172963, 3843574038, 332046372, 3779077468, 3439539464, 2417416507, 3120889907, 1289634270, 3685317313, 3169408221, 3982209785, 421179056, 652518988, 728374077, 1745520479, 3544291046]


Step: 0 ___________________________________________
val_loss: 64.16730499267578
winner_seed: 2910208350


Step: 1 ___________________________________________
val_loss: 97.4471206665039


Step: 2 ___________________________________________
val_loss: 75.50418090820312


Step: 3 ___________________________________________
val_loss: 79.84542083740234


Step: 4 ___________________________________________
val_los

In [87]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 124ms/step - loss: 9558.7627 - val_loss: 5441.2236
Epoch 2/10000
4/4 [==============================] - 0s 13ms/step - loss: 4279.2417 - val_loss: 1052.2495
Epoch 3/10000
4/4 [==============================] - 0s 11ms/step - loss: 305.7150 - val_loss: 203.9966
Epoch 4/10000
4/4 [==============================] - 0s 10ms/step - loss: 156.1534 - val_loss: 193.6104
Epoch 5/10000
4/4 [==============================] - 0s 9ms/step - loss: 133.5510 - val_loss: 291.2187
Epoch 6/10000
4/4 [==============================] - 0s 11ms/step - loss: 124.1385 - val_loss: 175.1456
Epoch 7/10000
4/4 [==============================] - 0s 12ms/step - loss: 106.3783 - val_loss: 166.7755
Epoch 8/10000
4/4 [==============================] - 0s 10ms/step - loss: 94.2934 - val_loss: 232.1576
Epoch 9/10000
4/4 [==============================] - 0s 10ms/step - loss: 109.3800 - val_loss: 145.7054
Epoch 10/10000
4/4 [==============================] - 0s 10ms

Epoch 80/10000
4/4 [==============================] - 0s 11ms/step - loss: 57.6037 - val_loss: 155.3763
Epoch 81/10000
4/4 [==============================] - 0s 12ms/step - loss: 56.1663 - val_loss: 136.2583
Epoch 82/10000
4/4 [==============================] - 0s 11ms/step - loss: 57.2770 - val_loss: 137.0590
Epoch 83/10000
4/4 [==============================] - 0s 10ms/step - loss: 54.7565 - val_loss: 162.8437
Epoch 84/10000
4/4 [==============================] - 0s 11ms/step - loss: 69.3883 - val_loss: 144.2100
Epoch 85/10000
4/4 [==============================] - 0s 11ms/step - loss: 60.3545 - val_loss: 146.8611
Epoch 86/10000
4/4 [==============================] - 0s 12ms/step - loss: 54.1296 - val_loss: 167.2194
Epoch 87/10000
4/4 [==============================] - 0s 12ms/step - loss: 66.9182 - val_loss: 154.5820
Epoch 88/10000
4/4 [==============================] - 0s 10ms/step - loss: 55.9325 - val_loss: 141.2916
Epoch 89/10000
4/4 [==============================] - 0s 11ms/st

4/4 [==============================] - 0s 13ms/step - loss: 38.8743 - val_loss: 114.1509
Epoch 159/10000
4/4 [==============================] - 0s 10ms/step - loss: 34.0145 - val_loss: 117.5006
Epoch 160/10000
4/4 [==============================] - 0s 11ms/step - loss: 36.7440 - val_loss: 114.1697
Epoch 161/10000
4/4 [==============================] - 0s 11ms/step - loss: 34.9220 - val_loss: 113.5406
Epoch 162/10000
4/4 [==============================] - 0s 11ms/step - loss: 33.8993 - val_loss: 129.2596
Epoch 163/10000
4/4 [==============================] - 0s 10ms/step - loss: 35.7668 - val_loss: 111.4824
Epoch 164/10000
4/4 [==============================] - 0s 10ms/step - loss: 34.0342 - val_loss: 114.1197
Epoch 165/10000
4/4 [==============================] - 0s 10ms/step - loss: 32.2051 - val_loss: 127.0027
Epoch 166/10000
4/4 [==============================] - 0s 10ms/step - loss: 38.8662 - val_loss: 110.0001
Epoch 167/10000
4/4 [==============================] - 0s 11ms/step - l

4/4 [==============================] - 0s 11ms/step - loss: 29.1165 - val_loss: 124.4902
Epoch 237/10000
4/4 [==============================] - 0s 11ms/step - loss: 31.2604 - val_loss: 117.4727
Epoch 238/10000
4/4 [==============================] - 0s 9ms/step - loss: 34.0744 - val_loss: 116.1832
Epoch 239/10000
4/4 [==============================] - 0s 11ms/step - loss: 28.5322 - val_loss: 124.8096
Epoch 240/10000
4/4 [==============================] - 0s 13ms/step - loss: 34.4867 - val_loss: 124.9388
Epoch 241/10000
4/4 [==============================] - 0s 11ms/step - loss: 34.2110 - val_loss: 115.0839
Epoch 242/10000
4/4 [==============================] - 0s 13ms/step - loss: 27.4701 - val_loss: 125.0550
Epoch 243/10000
4/4 [==============================] - 0s 12ms/step - loss: 30.0549 - val_loss: 114.8817
Epoch 244/10000
4/4 [==============================] - 0s 11ms/step - loss: 35.2419 - val_loss: 126.3487
Epoch 245/10000
4/4 [==============================] - 0s 10ms/step - lo

4/4 [==============================] - 0s 11ms/step - loss: 28.2415 - val_loss: 108.0947
Epoch 315/10000
4/4 [==============================] - 0s 12ms/step - loss: 28.8411 - val_loss: 107.6997
Epoch 316/10000
4/4 [==============================] - 0s 11ms/step - loss: 28.6811 - val_loss: 108.5312
Epoch 317/10000
4/4 [==============================] - 0s 10ms/step - loss: 28.5845 - val_loss: 105.6193
Epoch 318/10000
4/4 [==============================] - 0s 10ms/step - loss: 27.5218 - val_loss: 110.5585
Epoch 319/10000
4/4 [==============================] - 0s 10ms/step - loss: 28.3022 - val_loss: 113.0596
Epoch 320/10000
4/4 [==============================] - 0s 11ms/step - loss: 27.4051 - val_loss: 111.1928
Epoch 321/10000
4/4 [==============================] - 0s 12ms/step - loss: 29.4685 - val_loss: 111.6233
Epoch 322/10000
4/4 [==============================] - 0s 11ms/step - loss: 27.9364 - val_loss: 108.3297
Epoch 323/10000
4/4 [==============================] - 0s 11ms/step - l

...

4/4 [==============================] - 0s 10ms/step - loss: 9.4206 - val_loss: 122.4754
Epoch 2100/10000
4/4 [==============================] - 0s 11ms/step - loss: 8.9479 - val_loss: 123.8176
Epoch 2101/10000
4/4 [==============================] - 0s 13ms/step - loss: 7.5170 - val_loss: 120.6053
Epoch 2102/10000
4/4 [==============================] - 0s 11ms/step - loss: 7.6267 - val_loss: 119.0459
Epoch 2103/10000
4/4 [==============================] - 0s 12ms/step - loss: 9.0324 - val_loss: 117.3116
Epoch 2104/10000
4/4 [==============================] - 0s 12ms/step - loss: 8.8191 - val_loss: 120.8620
Epoch 2105/10000
4/4 [==============================] - 0s 10ms/step - loss: 9.4887 - val_loss: 115.9946
Epoch 2106/10000
4/4 [==============================] - 0s 10ms/step - loss: 8.3796 - val_loss: 117.8658
Epoch 2107/10000
4/4 [==============================] - 0s 11ms/step - loss: 7.4223 - val_loss: 119.6798
Epoch 2108/10000
4/4 [==============================] - 0s 11ms/step - l

4/4 [==============================] - 0s 11ms/step - loss: 8.7800 - val_loss: 125.2593
Epoch 2178/10000
4/4 [==============================] - 0s 11ms/step - loss: 9.6133 - val_loss: 124.2222
Epoch 2179/10000
4/4 [==============================] - 0s 10ms/step - loss: 8.4149 - val_loss: 125.7864
Epoch 2180/10000
4/4 [==============================] - 0s 11ms/step - loss: 9.1336 - val_loss: 125.7670
Epoch 2181/10000
4/4 [==============================] - 0s 12ms/step - loss: 9.6422 - val_loss: 124.2574
Epoch 2182/10000
4/4 [==============================] - 0s 10ms/step - loss: 8.4526 - val_loss: 123.4661
Epoch 2183/10000
4/4 [==============================] - 0s 10ms/step - loss: 8.4632 - val_loss: 127.2443
Epoch 2184/10000
4/4 [==============================] - 0s 13ms/step - loss: 7.8851 - val_loss: 126.1475
Epoch 2185/10000
4/4 [==============================] - 0s 11ms/step - loss: 8.3914 - val_loss: 122.3441
Epoch 2186/10000
4/4 [==============================] - 0s 10ms/step - l

4/4 [==============================] - 0s 12ms/step - loss: 9.6670 - val_loss: 126.8883
Epoch 2256/10000
4/4 [==============================] - 0s 12ms/step - loss: 8.1709 - val_loss: 130.0502
Epoch 2257/10000
4/4 [==============================] - 0s 10ms/step - loss: 8.7747 - val_loss: 126.7837
Epoch 2258/10000
4/4 [==============================] - 0s 10ms/step - loss: 7.9208 - val_loss: 127.1532
Epoch 2259/10000
4/4 [==============================] - 0s 10ms/step - loss: 8.5893 - val_loss: 127.2017
Epoch 2260/10000
4/4 [==============================] - 0s 12ms/step - loss: 6.7903 - val_loss: 126.8309
Epoch 2261/10000
4/4 [==============================] - 0s 11ms/step - loss: 9.3497 - val_loss: 128.0102
Epoch 2262/10000
4/4 [==============================] - 0s 11ms/step - loss: 8.7637 - val_loss: 127.5452
Epoch 2263/10000
4/4 [==============================] - 0s 10ms/step - loss: 7.8080 - val_loss: 129.4476
Epoch 2264/10000
4/4 [==============================] - 0s 12ms/step - l

4/4 [==============================] - 0s 10ms/step - loss: 9.1945 - val_loss: 104.4440
Epoch 2334/10000
4/4 [==============================] - 0s 10ms/step - loss: 9.5754 - val_loss: 108.9464
Epoch 2335/10000
4/4 [==============================] - 0s 10ms/step - loss: 8.1314 - val_loss: 123.1468
Epoch 2336/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.7572 - val_loss: 124.5467
Epoch 2337/10000
4/4 [==============================] - 0s 10ms/step - loss: 7.9817 - val_loss: 121.0795
Epoch 2338/10000
4/4 [==============================] - 0s 9ms/step - loss: 6.9451 - val_loss: 119.0189
Epoch 2339/10000
4/4 [==============================] - 0s 12ms/step - loss: 8.9623 - val_loss: 121.1323
Epoch 2340/10000
4/4 [==============================] - 0s 9ms/step - loss: 7.4970 - val_loss: 124.3039
Epoch 2341/10000
4/4 [==============================] - 0s 10ms/step - loss: 8.4686 - val_loss: 125.0579
Epoch 2342/10000
4/4 [==============================] - 0s 8ms/step - loss

4/4 [==============================] - 0s 12ms/step - loss: 6.2857 - val_loss: 122.1896
Epoch 2412/10000
4/4 [==============================] - 0s 11ms/step - loss: 7.0870 - val_loss: 121.5800
Epoch 2413/10000
4/4 [==============================] - 0s 12ms/step - loss: 10.0070 - val_loss: 124.2449
Epoch 2414/10000
4/4 [==============================] - 0s 12ms/step - loss: 6.9477 - val_loss: 124.4862
Epoch 2415/10000
4/4 [==============================] - 0s 11ms/step - loss: 6.6214 - val_loss: 122.3810
Epoch 2416/10000
4/4 [==============================] - 0s 11ms/step - loss: 6.5086 - val_loss: 124.4286
Epoch 2417/10000
4/4 [==============================] - 0s 10ms/step - loss: 7.7616 - val_loss: 125.6091
Epoch 2418/10000
4/4 [==============================] - 0s 12ms/step - loss: 8.7508 - val_loss: 125.2511
Epoch 2419/10000
4/4 [==============================] - 0s 11ms/step - loss: 7.7763 - val_loss: 120.3768
Epoch 2420/10000
4/4 [==============================] - 0s 10ms/step - 

In [88]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [89]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 15ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,97.409607,99.867462,100.363129,100.662292,97.397293,86.662857,93.674652,92.126389,91.252487,91.119736,91.114655,93.180527,95.240372,97.444626,99.29097,99.364914,96.358139,85.787949,92.430344,92.863205,91.924393,92.013168,92.809021,95.748489,97.575012,98.296661,98.728256,98.168137,99.250992,-0.894646
Target,86.678,100.174,89.777,94.81,93.446,87.54,96.706,74.086,92.676,82.865,71.18,108.826,105.694,107.106,101.893,110.551,100.587,94.22,111.377,98.614,94.022,91.546,95.674,88.927,105.688,104.711,97.53,99.651,76.71,75.456
Error,10.731606,0.306541,10.586128,5.852295,3.951294,0.877144,3.031349,18.04039,1.423515,8.254738,19.934654,15.64547,10.453629,9.661377,2.602028,11.186089,4.228859,8.432053,18.946655,5.750793,2.09761,0.467171,2.864983,6.821487,8.112991,6.414337,1.198257,1.482864,22.540993,76.350647


In [90]:
display(mae)
display(mape)

9.941599

0.11510746

In [91]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [92]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 16ms/step
Ano-0: |Prediction[[1134.831]] - Target[1078.7640000000001]| =  Error: [[56.067017]]; MAPE:[[0.05197338]]
1/1 [==============================] - 0s 17ms/step
Ano-0: |Prediction[[1131.2756]] - Target[1200.211]| =  Error: [[68.935425]]; MAPE:[[0.05743609]]
1/1 [==============================] - 0s 15ms/step
Ano-5: |Prediction[[491.1244]] - Target[559.746]| =  Error: [[68.62158]]; MAPE:[[0.12259415]]


[array([[56.067017]], dtype=float32),
 array([[68.935425]], dtype=float32),
 array([[68.62158]], dtype=float32)]

64.54134

0.07733454